In [1]:
print("in the name of GOD")

in the name of GOD


In [2]:
import re

In [3]:
FNV_prime_32 =  2^24 + 2^8 + 0x93
FNV_prime_64 = 2^40 + 2^8 + 0xb3
FNV_prime_128 = 2^88 + 2^8 + 0x3b

offset_basis_32 = 2166136261
offset_basis_64 = 14695981039346656037
offset_basis_128 = 144066263297769815596495629667062367629
#offset_basis_128 = 193

In [4]:
def fnv1a(hex_input, size, flag = 0):
    #Constants
    if size == 32:
        FNV_prime = FNV_prime_32
        offset_basis = offset_basis_32
        module = 2**32
    elif size == 64:
        FNV_prime = FNV_prime_64
        offset_basis = offset_basis_64
        module = 2**64
    elif size == 128:
        FNV_prime = FNV_prime_128
        offset_basis = offset_basis_128    
        module = 2**128    
    hash = offset_basis
    hex_input = hex_input.replace(" ", "")
    hex_input = re.findall('..', hex_input)
    for byte_of_data in hex_input:        
        #print(byte_data)
        if flag == 1:
            hash = mod(hash + int(byte_of_data, 16), 2**128)
        else:
            hash = hash ^^ int(byte_of_data, 16)
        hash = (hash * FNV_prime) % module
    return hash

In [5]:
fnv1a("8c2565b0f35411600c3c0e20e21235", 128)

314800192907571496543841540975179960854

In [6]:
def convert2hex(l):
    temp = map(hex, l)
    output = temp
    for i in range(len(l)):
        if len(temp[i]) < 2:
            output[i] = temp[i].zfill(2)
    output = "".join(output)
    return output

# LLL

In [7]:
n = 400
t = 200
g = FNV_prime_128
E = []
temp = [0 for _ in range(n)] + [t * 2**128]
E.append(temp)
f = lambda i, j: (i == j)*1
for i in range(n):
    temp = [f(i, j) for j in range(n)] + [mod(g**(n - i), 2**128)]    
    E.append(temp)
A = matrix(ZZ, n + 1, n + 1, E)
#pretty_print(A)
B = A.LLL()
B1 = []
for i in range(B.nrows()):
    if B[i, -1] == 0:
        B1.append(B.row(i))
#print(B1)

In [9]:
for selected_row in range(len(B1)):
    selected_coeffs = list(B1[selected_row])
    #selected_ceffs = map(Integer, ['0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '16', '0'])
    #selected_coeffs = B[selected_row, :]
    #print(B1[selected_row])
    tr = -1 * min(selected_coeffs)
    #tr = 59
    #print(tr)
    X = [tr + selected_coeffs[i] for i in range(n)]
    #print(X)
    Y = [tr for _ in range(n)]
    #print(Y)
    X = convert2hex(X)
    Y = convert2hex(Y)
    #print("%s = %s" % ("X", X))
    #print("%s = %s" % ("Y", Y))
    if fnv1a(X, 128, 1) == fnv1a(Y, 128, 1):
        print("%s = %s" % ("X", X))
        print("%s = %s" % ("Y", Y))
        print("%s = %s" % ("fnv1a(X)", (fnv1a(X, 128, 1))))
        print("%s = %s" % ("fnv1a(Y)", (fnv1a(Y, 128, 1))))

X = 06080608040204040606090302040707000600040203080507040605060406050a08050206090605030605070506040306050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505
Y = 05050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050

fnv1a(X) = 33260037918621555361504052869191972589
fnv1a(Y) = 33260037918621555361504052869191972589
X = 07060909090b0a0a0b08060d080d0607090a090a0a080a090a070b090a08050500050a0e0a0a0b060b0806070a080a0a08090a090b0a0a0a090a09080b090909080809090909090909090909090909090909090909090909090909090909090909090909090909090909090909090909090909090909090909090909090909090909090909090909090909090909090909090909090909090909090909090909090909090909090909090909090909090909090909090909090909090909090909090909090909090909090909090909090909090909090909090909090909090909090909090909090909090909090909090909090909090909090909090909090909090909090909090909090909090909090909090909090909090909090909090909090909090909090909090909090909090909090909090909090909090909090909090909090909090909090909090909090909090909090909090909090909090909090909090909090909090909090909090909090909090909090909090909090909090909090909090909
Y = 0909090909090909090909090909090909090909090909090909090909090909090909090909090909090909090

fnv1a(Y) = 142540504217339914037898382482723350733
X = 070906060108020405040607090808050b0106080808040407060007080b06080807050707020408050608050708080806050606060506060606060606060606060606060606060606070606060606060606060606060606060606060606060606060606060606060606060606060606060606060606060606060606060606060606060606060606060606060606060606060606060606060606060606060606060606060606060606060606060606060606060606060606060606060606060606060606060606060606060606060606060606060606060606060606060606060606060606060606060606060606060606060606060606060606060606060606060606060606060606060606060606060606060606060606060606060606060606060606060606060606060606060606060606060606060606060606060606060606060606060606060606060606060606060606060606060606060606060606060606060606060606060606060606060606060606060606060606060606060606060606060606060606060606060606
Y = 06060606060606060606060606060606060606060606060606060606060606060606060606060606060606060606060606060606060606060606060606060606060606060606

X = 030902080603050809070a09060502070609030503080502070709060b0804060704080708060204020600040806040406060906070504060606070606060506060406060606060606060606060606060606060606060606060606060606060606060606060606060606060606060606060606060706060606060606060606060606060606060606060606060606060606060606060606060606060606060606060606060606060606060606060606060606060606060606060606060606060606060606060606060606060606060606060606060606060606060606060606060606060606060606060606060606060606060606060606060606060606060606060606060606060606060606060606060606060606060606060606060606060606060606060606060606060606060606060606060606060606060606060606060606060606060606060606060606060606060606060606060606060606060606060606060606060606060606060606060606060606060606060606060606060606060606060606060606060606060606
Y = 06060606060606060606060606060606060606060606060606060606060606060606060606060606060606060606060606060606060606060606060606060606060606060606060606060606060606060606060606060606060606060606060

X = 05070604020507040605050405050605040406050504070a05030800030204060507030506080609080405020303060406040506040507050506050506050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050605050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505
Y = 05050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050

...

fnv1a(Y) = 142540504217339914037898382482723350733
X = 0905010708060407030a040705020008040707060302030405060703030407050509060101010504040609070607030406070405040606030605050505050406040505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050506050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505
Y = 05050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505050505

X = 06030604030703010102060101030403080107050406050405050403040405040102060002050308030207020405050304030404040306040305030403040504040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040504040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404
Y = 04040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040

X = 07020100070705030206000304040306030205040501020504020104060205040103040202030904060405050102060205030504040203030505040504040404040304040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404050404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404
Y = 04040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040

Y = 04040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404040404
fnv1a(X) = 142540504217339914037898382482723350733
fnv1a(Y) = 142540504217339914037898382482723350733
X = 03030307090105020201040306020105020708020304030500060706020501050607030304020301020105020

X = 03030005050300030205020101040305000605040403030000040505020104050403030401010301020506040504050403020204020204030303040302030402030303030303030303030303030303030303030303030303030303030303030303030303030303030303030303030303030303030303030303030303030303030303030303030303030303030303030303030303030303030303030303030303030303030303030303030303030303030303030303030303030303030303030303030303030303030303030303030303030303030303030303030303030303030303030303030303030303030303030303030303030303030303030303030303030303030303030303030303030303030303030303030303030303030303030303030303030303030303030303030303030303030303030303030303030303030303030303030303030303030303030303030303030303030303030303030303030303030303030303030303030304030303030303030303030303030303030303030303030303030303030303030303
Y = 03030303030303030303030303030303030303030303030303030303030303030303030303030303030303030303030303030303030303030303030303030303030303030303030303030303030303030303030303030303030303030303030